# Data analysis for different reference orientations

This notebook changes the reference orientation for the calculation of lattice msiorientation from EBSD data. The reference orientations are:
    - GROD Mean: The grain mean orientation (GROD Mean)
    - GROD Mean Min: The pixel in the grain with the minimum misorientation as calculaed by GROD Mean
    - GROD Mean Max: The pixel in the grain with the maximum misorientation as calculaed by GROD Mean
    - GROD Centroid: The pixel at the EUclidean centre of the grain
    - GROD Lowest KAM: The pixel with the lowest Kernal Avergae Misorientation value
NB: To calculate the pixel misorientation with respect to the original grain orientation (prior to deformation), see the Linker class and the associated notebook (see Michael Atkinson)

## Import modules

In [306]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import transform as tf
from skimage import io

import hrdic
import ebsd

import copy

%matplotlib osx

## Load DIC data

In [307]:
#Load in DIC map
DicFilePath = "./example_data_AH/"
DicMap = hrdic.Map(DicFilePath, "hrdicData.txt")

Loaded DaVis 8.1.5 data (dimensions: 1860 x 1628 pixels, sub-window size: 8 x 8 pixels)


## Load EBSD Data

In [308]:
mapFiles = ["./example_data_AH/ebsdData"]

maps = []

for i, mapFile in enumerate(mapFiles):
    maps.append(ebsd.Map(mapFile, "cubic"))
    maps[i].loadData(mapFile, "cubic")
    maps[i].binData = maps[i].binData[::-1]  #rotate the map 180 degrees
    maps[i].buildQuatArray()
    maps[i].findBoundaries(boundDef = 6)
    maps[i].findGrains(minGrainSize=10)
    maps[i].calcGrainMisOri(calcAxis = True)

Loaded EBSD data (dimensions: 612 x 465 pixels, step size: 0.5 um)
Error with cpr file. Number of phases mismatch.
Loaded EBSD data (dimensions: 612 x 465 pixels, step size: 0.5 um)


In [14]:
maps[0].plotMisOriMap()

## Record the original grain reference orientations

In [309]:
def recordGrainOrigRefOri(self):
    self.origRefOri = copy.deepcopy(self.refOri)

ebsd.Grain.recordGrainOrigRefOri=recordGrainOrigRefOri

In [310]:
# test the function
maps[0].grainList[0].recordGrainOrigRefOri()
print(maps[0].grainList[0].origRefOri)

[0.6726, -0.2929, -0.0118, -0.6795]


In [311]:
# record original reference oriengtation for each grain
for grain in maps[0].grainList:
    grain.recordGrainOrigRefOri()

## Revert back to the original grain reference orientations

In [312]:
def revertGrainRefOriToOriginalRefOris(self):
    self.refOri = copy.deepcopy(self.origRefOri)

ebsd.Grain.revertGrainRefOriToOriginalRefOris=revertGrainRefOriToOriginalRefOris

In [326]:
# revert grain reference ori to original reference ori
for grain in maps[0].grainList:
    grain.revertGrainRefOriToOriginalRefOris()

In [327]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

In [330]:
maps[0].plotMisOriMap(vmax=3,plotGBs=True)

## Link the DIC and EBSD data

In [315]:
# load the disMap crop parameters
DicMap.cropDists = np.loadtxt("./example_data_AH/dicMapCropDists.txt")

# set the dic map crop parameters
DicMap.cropDists_temp=[]
for p,q in DicMap.cropDists:
    p,q = int(p),int(q)
    DicMap.cropDists_temp.append([p,q])
DicMap.cropDists_temp=np.array(DicMap.cropDists_temp)
DicMap.cropDists=copy.deepcopy(DicMap.cropDists_temp)

# define the crop
DicMap.setCrop(xMin=DicMap.cropDists[0,0], xMax=DicMap.cropDists[0,1], yMin=DicMap.cropDists[1,0], yMax=DicMap.cropDists[1,1])

# load and define the homogPoints
DicMap.homogPoints = np.loadtxt("./example_data_AH/dicMapHomogPoints.txt")
maps[0].homogPoints = np.loadtxt("./example_data_AH/ebsdMapHomogPoints.txt")

# link the dic and ebsd data
DicMap.linkEbsdMap(maps[0])

# find the grains
DicMap.findGrains(minGrainSize=1434)

In [67]:
DicMap.locateGrainID()

Grain ID: 32


In [73]:
# test the link has worked
grainNum = DicMap.currGrainId
DicMap.grainList[grainNum].plotMaxShear()
maps[0].grainList[DicMap.ebsdGrainIds[grainNum]].plotMisOri()

## Plot grain-to-grain DIC-EBSD correlation

In [316]:
# Define the DicMap pixel resolution in order to obtain grain size information
DicMap.pixelResolution = 0.117 # um

In [317]:
# plot data correlation function
def plotCorrelation(dicSelf,ebsdSelf,grainDiameter=False):
    plt.figure()
    for i,grain in enumerate(dicSelf):
        x = np.nanmean(grain.maxShearList)
        y = np.nanmean((np.arccos(ebsdSelf.grainList[dicSelf.ebsdGrainIds[i]].misOriList))*180/np.pi)
        if grainDiameter:
            area = len(grain.maxShearList)
            diameter = ((area/np.pi)**0.5)*2*dicSelf.pixelResolution # um
            plt.scatter(x,y,c=diameter,s=diameter,vmin=20,vmax=40,cmap="viridis")
        else:
            plt.scatter(x,y,c="grey")
    if grainDiameter:    
        plt.colorbar(label="Grain diameter [$\\mu$ m]", shrink=0.8)
    plt.xlabel("HRDIC strain, $\epsilon_{eff}$ [ ]")
    plt.ylabel("EBSD misorientation, $\phi$ [ $^o$ ]")

In [109]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Change the reference orientation for the ebsd misOri calculation

#### Best Band Contrast value

In [318]:
# define the BC map
def generateBCMap(self):
    self.bcmap = np.reshape(self.binData[('IB2')], (self.yDim, self.xDim))
    
ebsd.Map.generateBCMap=generateBCMap

In [319]:
maps[0].generateBCMap()

#### Change refOri to that of the pixel with best BC

In [320]:
def refOribestBC(self,ebsdSelf=maps[0]):

    # create new refOri
    grainBcList = []
    for coord in self.coordList:
        bcVal = ebsdSelf.bcmap[coord[1],coord[0]]
        grainBcList.append(bcVal)
    maxBcIdx = np.argmax(grainBcList)
    newQuat = self.quatList[maxBcIdx]
    self.refOri = copy.deepcopy(newQuat) 

ebsd.Grain.refOribestBC=refOribestBC

In [321]:
for grain in maps[0].grainList:
    grain.refOribestBC()

In [322]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

Calculating grain misorientations...

/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in true_divide
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in arccos
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in sqrt
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))


In [323]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Change back to original refOri

In [146]:
# revert grain reference ori to original reference ori
for grain in maps[0].grainList:
    grain.revertGrainRefOriToOriginalRefOris()

In [147]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

In [148]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Calculate misOri wrt. mean-min misOri

In [151]:
def refOriMeanMin(self):
    # create new refOri
    minIdx = np.argmax(self.misOriList) # use np.argmax as minMisOri inversely proportional to arccos(minMisOri)
    newQuat = self.quatList[minIdx]
    self.refOri = copy.deepcopy(newQuat) 
ebsd.Grain.refOriMeanMin=refOriMeanMin

In [152]:
for grain in maps[0].grainList:
    grain.refOriMeanMin()

In [154]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

Calculating grain misorientations...

/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in true_divide
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in arccos
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in sqrt
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))


In [155]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Change back to original refOri

In [156]:
# revert grain reference ori to original reference ori
for grain in maps[0].grainList:
    grain.revertGrainRefOriToOriginalRefOris()

In [159]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

In [160]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Calculate misOri wrt. mean-max misOri

In [331]:
def refOriMeanMax(self):
    # create new refOri
    maxIdx = np.argmin(self.misOriList) # use np.argmin as maxMisOri inversely proportional to arccos(maxMisOri)
    newQuat = self.quatList[maxIdx]
    self.refOri = copy.deepcopy(newQuat) 
ebsd.Grain.refOriMeanMax=refOriMeanMax

In [332]:
for grain in maps[0].grainList:
    grain.refOriMeanMax()

In [333]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

Calculating grain misorientations...

/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in arccos
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in sqrt
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in true_divide
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))


In [164]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Change back to original refOri

In [165]:
# revert grain reference ori to original reference ori
for grain in maps[0].grainList:
    grain.revertGrainRefOriToOriginalRefOris()

In [166]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

In [167]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Calculate misOri wrt. pixel with the lowest KAM

#### Calculate map KAM

In [183]:
def calcKamNxN(self,n=3,maxMisOri=5, plotKamData=False,plotGBs=False,localGrad=False,boundaryColour='black'):
    
    if n <= 1:
        raise ValueError("Please enter a kernel dimenion n, where n > 1 and n is an odd number")
    oddCheck = n%2
    if oddCheck == 0:
        raise ValueError("Please enter a kernel dimenion n, where n > 1 and n is an odd number")

    # separate the quat coefficients
    quatComps = np.empty((4, self.yDim, self.xDim))
    for i, row in enumerate(self.quatArray):
        for j, quat in enumerate(row):
            quatComps[:, i, j] = quat.quatCoef
    
    # define the data as the separated quat coefficients
    data = quatComps

    # define r, the remainder, to define the boundary regions
    r=int((n-1)/2)

    # kamData=np.empty((data.shape[1]-r,data.shape[2]-r))
    kamData=np.zeros((np.shape(data)[1],np.shape(data)[2])) # initialise with zeroes so that the final kamMap is the same size as the data

    for i in range(r,np.shape(data)[1]-r): # avoid boundaries, r
        for j in range(r,np.shape(data)[2]-r):

            # initialise a data kernel
            dataKernel=np.zeros((4,n,n))
            dataKernelList=np.zeros((4,n*n))
            
            centralPixelIndex=int((n-1)/2)
            cPixIdx=np.full((2,1),centralPixelIndex)
            
            distList=[]

            # grab the quat data for the data kernel
            for q in range(np.shape(data)[0]):

                # define the data kernel
                for row in range(n):
                    for column in range(n):                            
                        dataKernel[q,row,column] = data[q,i-r+row,j-r+column]
                        
                        if localGrad:
                        
                            distX = abs(row-cPixIdx[0])
                            distY = abs(column-cPixIdx[1])
                            dist = ((distX**2)+(distY**2))**0.5
                            distList.append(dist.tolist())
                            
                        else:
                            dist=1
                            distList.append([dist])
                            
                dataKernelList[q,:]=dataKernel[q,:,:].flatten()
            
            distList_temp=[]
            for dist in distList:
                for k in dist:
                    distList_temp.append(k)
            distList=distList_temp
            

            #  find the mirOri between the central data pixel and the data kernel pixels
            misOriKernel=abs(np.einsum("i,ij->j",data[:,i,j],dataKernelList[:]))
            # convert to degrees
            misOriKernelDeg=2*np.arccos(misOriKernel)*180/np.pi

            # replace misOris > maxMisOri with NaN
            misOriKernelDeg_new=np.empty(n*n)
            for val in range(len(misOriKernelDeg_new)):
                if misOriKernelDeg[val] >= maxMisOri:
                    newVal=np.nan
                else:
                    newVal=misOriKernelDeg[val]
                misOriKernelDeg_new[val]=newVal

            # replace central value with NaN
            centralPixelIndex=int(((n*n)-1)/2)
            misOriKernelDeg_new[centralPixelIndex] = np.nan
            
            gradList=[]
            for dif,dist in zip(misOriKernelDeg_new,distList):
                if np.isnan(dif):
                    gradList.append(np.nan)
                else:
                    gradVal = dif/dist
                    gradList.append(gradVal)
                    
            
            # find mean of all misOris in kernel
            meanMisOri=np.nanmean(gradList)    

            # place misOri in kamData
            kamData[i,j] = meanMisOri

    # Plotting
    if plotKamData:
        plt.figure()
        plt.imshow(kamData,vmin=0,vmax=1)
        plt.colorbar(shrink=0.8,label="KAM [ $^o$ ]")

        if plotGBs:
            self.plotGBs(colour=boundaryColour)

    self.kamData=kamData

ebsd.Map.calcKamNxN=calcKamNxN

In [172]:
maps[0].calcKamNxN()

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:92: RuntimeWarning: Mean of empty slice


#### Plot KAM without re-calculating KAM map

In [220]:
def plotKAMMap(self,plotGBs=False,boundaryColour='black'):
    
    plt.figure()
    plt.imshow(self.kamData,vmin=0,vmax=1)
    plt.colorbar(shrink=0.8,label="KAM [ $^o$ ]")
        
    if plotGBs:
        self.plotGBs(colour=boundaryColour)
        
ebsd.Map.plotKAMMap=plotKAMMap

In [221]:
maps[0].plotKAMMap(plotGBs=True)

#### Find grain KAM values

In [188]:
def getGrainKam(self,kamMap=maps[0].kamData):
    kamList = []
    for coord in self.coordList:
        kamVal = kamMap[coord[1],coord[0]]
        kamList.append(kamVal)
    self.kamList=kamList
ebsd.Grain.getGrainKam=getGrainKam

In [189]:
for grain in maps[0].grainList:
    grain.getGrainKam()

#### Plot grain KAM

In [218]:
def plotKAM(self):
    
    plt.figure()
    x0, y0, xmax, ymax = self.extremeCoords
#     x0, y0, xmax, ymax = int(x0), int(y0), int(xmax), int(ymax)
    plotArea = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)
    plotData = self.kamList
    coord0s,coord1s=[],[]
    for coord0,coord1 in self.coordList:
        coord0s.append(coord0)
        coord1s.append(coord1)
    for coord0,coord1,data in zip(coord0s,coord1s,plotData):
        plotArea[coord1 - y0, coord0 - x0] = data
    plt.imshow(plotArea, vmin=0, vmax=0.3, cmap="viridis")
    
ebsd.Grain.plotKAM=plotKAM

In [219]:
maps[0].grainList[32].plotKAM()

#### Set grain refOri to pixel with lowest KAM, re-calculate misOri

In [222]:
def refOriLowestKAM(self):
    minKamValIndx = np.argmin(self.kamList)
    newQuat = self.quatList[minKamValIndx]
    self.refOri = copy.deepcopy(newQuat)   
ebsd.Grain.refOriLowestKAM=refOriLowestKAM

In [230]:
for grain in maps[0].grainList:
    grain.refOriLowestKAM()

In [231]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

Calculating grain misorientations...

/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in true_divide
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in arccos
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in sqrt
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))


In [232]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Change back to original refOri

In [236]:
# revert grain reference ori to original reference ori
for grain in maps[0].grainList:
    grain.revertGrainRefOriToOriginalRefOris()

In [237]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

In [235]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Calculate refOri as the grain centroid

#### Calculate GB proxigram

In [241]:
def calcProxiGram(self):
    indexBoundaries = []
    for index, value in np.ndenumerate(self.boundaries):
        if value == -1:
            indexBoundaries.append(index)

    proxBoundaries = np.copy(self.boundaries[:,:])

    numTrials = 1000
    trialDistances = np.ones((numTrials+1, proxBoundaries.shape[0], proxBoundaries.shape[1]))*1000

    # array of x and y coordinate of each pixel in the map
    coords = np.zeros((2, proxBoundaries.shape[0], proxBoundaries.shape[1]))
    for index, value in np.ndenumerate(proxBoundaries):
        coords[0, index[0], index[1]] = index[0]
        coords[1, index[0], index[1]] = index[1]

    # loop over each boundary point (p) and calcuale distance from p to all points in the map
    # once numTrails have been made the minimum of them will be found and stored
    j=1
    for indexBoundary in indexBoundaries:    
        trialDistances[j] = np.sqrt((coords[0] - indexBoundary[0])**2 + (coords[1] - indexBoundary[1])**2)

        if j==numTrials:
            trialDistances[0] = trialDistances.min(axis=0)
            j=0
        j+=1

    # find minimum distance to a boundary
    minDistances = trialDistances.min(axis=0)
    
    self.GBProxigram=minDistances

ebsd.Map.calcProxiGram=calcProxiGram

In [242]:
maps[0].calcProxiGram()

#### Plot proxigram map

In [274]:
def plotGBProxiGramMap(self):
    plt.figure()
    plt.imshow(self.GBProxigram)
    plt.colorbar(label="Proximity to nearest GB [$\\mu$ m]",shrink=0.8)

ebsd.Map.plotGBProxiGramMap=plotGBProxiGramMap

In [275]:
maps[0].plotGBProxiGramMap()

#### Get grain proxigram

In [281]:
def getGrainGBProxiGram(self,GBProxiGramMap=maps[0].GBProxigram):
    GBProxiGramList = []
    for coord in self.coordList:
        GBProxiGramVal = GBProxiGramMap[coord[1],coord[0]]
        GBProxiGramList.append(GBProxiGramVal)
    self.GBProxiGramList=GBProxiGramList

ebsd.Grain.getGrainGBProxiGram=getGrainGBProxiGram

In [282]:
for grain in maps[0].grainList:
    grain.getGrainGBProxiGram()

#### Plot proxigram for a grain

In [291]:
def plotGBProxigram(self):    
    plt.figure()
    x0, y0, xmax, ymax = self.extremeCoords
#     x0, y0, xmax, ymax = int(x0), int(y0), int(xmax), int(ymax)
    plotArea = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)
    plotData = self.GBProxiGramList
    coord0s,coord1s=[],[]
    for coord0,coord1 in self.coordList:
        coord0s.append(coord0)
        coord1s.append(coord1)
    for coord0,coord1,data in zip(coord0s,coord1s,plotData):
        plotArea[coord1 - y0, coord0 - x0] = data
    plt.imshow(plotArea, cmap="viridis")
    plt.axis("off")
    plt.colorbar(label="Proximity to nearest GB [$\\mu$ m]",shrink=0.8)
    
ebsd.Grain.plotGBProxigram=plotGBProxigram

In [292]:
maps[0].grainList[32].plotGBProxigram()

#### refOri as that of central pixel

In [296]:
def refOriGreatestGBDistance(self):
    # find max distance pixel and set to new quat
    maxDistanceIndx = np.argmax(self.GBProxiGramList)
    newQuat = self.quatList[maxDistanceIndx]
    self.refOri = copy.deepcopy(newQuat)

ebsd.Grain.refOriGreatestGBDistance=refOriGreatestGBDistance

In [303]:
for grain in maps[0].grainList:
    grain.refOriGreatestGBDistance()

In [304]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

Calculating grain misorientations...

/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in arccos
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in sqrt
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))
/Users/mclijah5/Dropbox (Research Group)/Allan/Macbook_2/Python/DataAnalysisUtilities/ebsd.py:636: RuntimeWarning: invalid value encountered in true_divide
  misOriAxis[:, :] = (2 * Dq[1:4, :] * np.arccos(Dq[0, :])) / np.sqrt(1 - np.power(Dq[0, :], 2))


In [305]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Change back to original refOri

In [300]:
# revert grain reference ori to original reference ori
for grain in maps[0].grainList:
    grain.revertGrainRefOriToOriginalRefOris()

In [301]:
# re-calculate misOri
maps[0].calcGrainMisOri(calcAxis = True)

In [302]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)